<!--<badge>--><a href="https://colab.research.google.com/github/huggingface/workshops/blob/main/luzern-university/03-gradio-demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

# Creating a Transformers demo with Gradio

## References

* https://huggingface.co/blog/gradio-spaces
* https://huggingface.co/blog/gradio

In [ ]:
!pip install transformers gradio sentencepiece

     |████████████████████████████████| 3.4 MB 25.8 MB/s 
     |████████████████████████████████| 981 kB 38.8 MB/s 
     |████████████████████████████████| 1.2 MB 19.7 MB/s 
     |████████████████████████████████| 895 kB 11.6 MB/s 
     |████████████████████████████████| 3.3 MB 29.6 MB/s 
     |████████████████████████████████| 61 kB 361 kB/s 
     |████████████████████████████████| 596 kB 40.7 MB/s 
     |████████████████████████████████| 206 kB 41.2 MB/s 
     |████████████████████████████████| 2.0 MB 30.5 MB/s 
     |████████████████████████████████| 961 kB 26.4 MB/s 
     |████████████████████████████████| 3.6 MB 21.1 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=23ff2fe62272592c58a67a515588bc20f1e9bdd7e9d4470580c4f3d66231abe0
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_

## Example 1: Using the Transformers pipeline

In [ ]:
import gradio as gr
from transformers import pipeline

In [ ]:
pipe = pipeline("text-classification", model="lewtun/xlm-roberta-base-finetuned-marc-en")

Downloading:   0%|          | 0.00/976 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
pipe("The Lord of the Rings is waaay too long to read!!")

[{'label': 'poor', 'score': 0.4347412586212158}]

In [ ]:
label2emoji = {"terrible": "💩", "poor": "😾", "ok": "🐱", "good": "😺", "great": "😻"}

def predict(text):
    preds = pipe(text)[0]
    return label2emoji[preds["label"]], round(preds["score"], 5)

predict("I love this soccer ball")

('😻', 0.82918)

In [ ]:
gradio_ui = gr.Interface(
    fn=predict,
    title="Predicting review scores from customer reviews",
    description="Enter some review text about an Amazon product and check what the model predicts for it's star rating.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["My favourite book is Cryptonomicon!"], ["私の好きな本は「クリプトノミコン」です"]
    ],
)

gradio_ui.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://21684.gradio.app

This share link will expire in 72 hours. To get longer links, send an email to: support@gradio.app


## Example 2: Using the Inference API from the Hugging Face Hub

In [ ]:
from huggingface_hub import InferenceApi

text = "My favourite book is Cryptonomicon!"
inference = InferenceApi("lewtun/xlm-roberta-base-finetuned-marc-en")
preds = inference(inputs=text)
preds[0]

In [ ]:
sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True) 
sorted_preds

In [ ]:
def inference_predict(text):
    inference = InferenceApi("lewtun/xlm-roberta-base-finetuned-marc-en")
    preds = inference(inputs=text)
    sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True)[0]
    return label2emoji[sorted_preds["label"]], round(sorted_preds["score"], 5)

In [ ]:
inference_predict(text)

In [ ]:
gradio_ui = gr.Interface.load(
    name="lewtun/xlm-roberta-base-finetuned-marc",
    src="huggingface",
    fn=inference_predict,
    title="Review analysis",
    description="Enter some text and check if model detects it's star rating.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["My favourite book is Cryptonomicon!"], ["私の好きな本は「クリプトノミコン」です"]
    ],
)

gradio_ui.launch(debug=True)